# Data collection and data pre-processing

In [ ]:
import pandas as pd

In [1]:
def get_13f_filings(start=0):
    start=0
    x=0
    while start<100000:
            print(f"Getting next 13F batch starting at {start}") 
            query={
                "query": { "query_string": { 
                    "query": "formType:\"13F-HR\" AND NOT formType:\"13F-HR/A\" AND periodOfReport:\"2021-12-31\"" 
                    } },
                "from": start,
                "size": "250",
                "sort": [{ "filedAt": { "order": "desc" } }]
            }
            response = queryApi.get_filings(query)
            if x==0:
                filings_batch=pd.json_normalize(response['filings'])
            else:
                z=pd.json_normalize(response['filings'])
                if (z.empty):
                    break
                filings_batch=pd.concat((filings_batch, z), axis = 0,ignore_index=True)
                #filings_batch=filings_batch.append(z,ignore_index = True)
            x=x+1
            start=start+200
            
    return filings_batch

def split_dataframe(df, chunk_size = 1000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

def Extracting_holdings(Filing):
    #filings_batch=Filing
    Filing=Filing[Filing['holdings'].notna()].reset_index(drop=True)
    for x in range(len(Filing)):  
        holdings_ = pd.json_normalize(Filing['holdings'][x])
        holdings_['id'] = Filing['id'][x]
        holdings_['cik'] = Filing['cik'][x]
        holdings_['companyName']=Filing['companyName']
        if x==0:
            Holdings= pd.DataFrame(columns=holdings_.columns)
        else:
            Holdings=pd.concat((Holdings, holdings_), axis = 0,ignore_index=True)
    return Holdings

def Holdings(df):
    for x in range(len(df)):
        print(x)
        Hold_try=Extracting_holdings(df[x])
        if x==0:
            Hold_final=Hold_try
        else:
            Hold_final=pd.concat((Hold_final, Hold_try), axis = 0,ignore_index=True)
            #Hold_final=Hold_final.append(Hold_try,ignore_index = True)
    return Hold_final

def cik_(df):
    for x in range(len(df)):
        print(x)
        w=df[x].reset_index()
        for y in range(len(df[x])):
            a=pd.DataFrame((w["cik"][y]).split(":"))
            z=a.transpose()
            if y==0:
                cik=z
            else:
                cik=pd.concat((cik, z), axis = 0,ignore_index=True)
                #cik=cik.append(z, ignore_index=True)
        if x==0:
            cik_final=cik
        else:
            cik_final=pd.concat((cik_final, cik), axis = 0,ignore_index=True)
            #cik_final=cik_final.append(cik, ignore_index=True)
    return cik_final



In [ ]:
def get_13f_HRA_filings(start=0):
    start=0
    x=0
    while start<100000:
            print(f"Getting next 13F batch starting at {start}") 
            query={
                "query": { "query_string": { 
                    "query": "formType:\"13F-HR/A\" AND periodOfReport:\"2021-12-31\"" 
                    } },
                "from": start,
                "size": "250",
                "sort": [{ "filedAt": { "order": "desc" } }]
            }
            response = queryApi.get_filings(query)
            if x==0:
                filings_batch=pd.json_normalize(response['filings'])
            else:
                z=pd.json_normalize(response['filings'])
                if (z.empty):
                    break
                    
                filings_batch=filings_batch.append(z,ignore_index = True)
            x=x+1
            start=start+200
            
    return filings_batch

# Fetching data from "www.sec.gov"

Write your email address in the place of 'abc@gmail.com' in statement: headers = {'User-Agent': 'abc@gmail.com'}

In [3]:
def fetch_13hra(link):
    conn = http.client.HTTPSConnection("www.sec.gov")
    payload = ''
    headers = {'User-Agent': 'abc@gmail.com'}
    conn.request("GET", link, payload, headers)
    res = conn.getresponse()
    data = res.read()
    c=data.decode("utf-8")
    df = pd.read_xml(c)
    return df

def fetch_url(Filings_HR_A):
    for x in range(len(Filings_HR_A)):  
        XML_url = pd.json_normalize(Filings_HR_A["documentFormatFiles"][x])
        XML_url['cik'] = Filings_HR_A['cik'][x]
        XML_url["filedAt"]=Filings_HR_A["filedAt"][x]
        if x==0:
            url_table= pd.DataFrame(columns=XML_url.columns)
            url_table=XML_url
        else:
            url_table=pd.concat((url_table,XML_url), axis=0, ignore_index=True)
           #url_table=url_table.append(XML_url,ignore_index = True)
    url_tbl_final=url_table[(url_table["type"]=="INFORMATION TABLE") & (url_table["sequence"]=="2")].iloc[1::2, :]
    return url_tbl_final
        
def fetch_data_from_xml(url_tbl_final):
    for x in range(len(url_tbl_final)):
        link=url_tbl_final["documentUrl"].iloc[x]
        df=fetch_13hra(link)
        df["cik"]=url_tbl_final["cik"].iloc[x]
        if x==0:
            hr_a_filings=df
        else:
            hr_a_filings=pd.concat((hr_a_filings, df), axis = 0,ignore_index=True)
    return hr_a_filings

# Regular expressions to correct the names for joining with CIK table

In [ ]:
def transformation_v1(df,x):
    df[x]=df[x].str.upper()
    df[x]=df[x].replace(to_replace =',|\.|\/|-|', value = '', regex = True).str.strip()   
    df[x]=df[x].replace('HOLDINGS|HOLDING|HLDGS','HLDG', regex=True)
    df[x]=df[x].replace('INCORPORATED|INC','IN', regex=True)
    df[x]=df[x].replace('INDUSTRIES|INSTRS|INDUSTRY','IND', regex=True)
    df[x]=df[x].replace('COMPANY|COS','CO', regex=True)
    df[x]=df[x].replace('CORPORATION','CORP', regex=True)
    df[x]=df[x].replace('GROUP|GROUPS|GRPS','GRP', regex=True)
    df[x]=df[x].replace('FUNDS|FUND','FDS', regex=True)
    df[x]=df[x].replace('TRUST|TRUSTS','TR', regex=True)
    #df[x]=df[x].replace('INDUSTRIES|INDUSTRY','INSTRS', regex=True)
    df[x]=df[x].replace('SOUTHERN|SOUTHN','STHN', regex=True)
    df[x]=df[x].replace('BOND|BONDS','BD', regex=True)
    df[x]=df[x].replace('WHOLESALE','WHSL', regex=True)
    df[x]=df[x].replace('LABORATORIES|LABORATORY','LABS', regex=True)
    df[x]=df[x].replace('BANK','BK', regex=True)
    df[x]=df[x].replace('EXCHANGE','EXCH', regex=True)
    return df[x]

def transformation_v2(df,x):
    df[x]=df[x].replace(to_replace =',|\.|\/|-|COM|IRELAND|(THE)|(NEW)|LP|US LF ETF|COM CL A|SHS|CL|ADR UK UNITED|ADR|IE IRELAND|CH|SWITZERLAND|SPONS ADR|SHSISIN#BMG4593F1041|ORD SHS|SA BEARER AND ADR|ETF|EFT|UTD UK', value = '', regex = True).str.strip()
    # Hold_final[x].replace('INCORPORATED','INC')
    df[x]=df[x].replace('COS','CO', regex=True).str.strip()
    df[x]=df[x].replace('CAPITAL','CAP', regex=True).str.strip()
    df[x]=df[x].replace('INTERNATIONAL','INTL', regex=True).str.strip()
    df[x]=df[x].replace('INCS','INC', regex=True).str.strip()
    df[x]=df[x].replace('TECHNOLOGY|TECHNOLOGIES','TECH', regex=True).str.strip()
    return df[x]

def transformation_v3(df,x):
    df[x]=df[x].replace(to_replace =',|\.|\/|-|COM|IRELAND|CO|CORP|INC|IN|IN ASS A|S&P 500|IN  USD001  A| S&P 500 VALUE|EA REPR 5 ORD US|LIMITED|LTD|IN B|SPON|IN A|EUR001|', value = '', regex = True).str.strip()
    df[x]=df[x].replace('SYSTEMS|SYSTEM','SYS', regex=True).str.strip()
    df[x]=df[x].replace('WORKS','WKS', regex=True).str.strip()
    df[x]=df[x].replace('NATIONAL','NATL', regex=True).str.strip()
    df[x]=df[x].replace('FINANCIAL','FINL', regex=True).str.strip()
    return df[x]

# Creation of Nodes and Relationships for graph creation

In [7]:
def Nodes_(Holdings,Filings):
    import pandas as pd
    Nodes=Filings[["companyName_unchanged","cik"]].drop_duplicates()
    append=Holdings[["nameOfIssuer_unchanged","CIK"]].drop_duplicates()
    Nodes.columns=["companyName","cik"]
    append.columns=["companyName","cik"]
    
    Nodes=pd.concat((Nodes,append), axis=0,ignore_index=True)
    Nodes=Nodes.drop_duplicates()
    
    Nodes['cik']=Nodes["cik"].astype(int).astype(str)
    Nodes['cik']=Nodes['cik'].str.lstrip("0").str.rstrip("0")
    
    Minus=Nodes[Nodes.duplicated('cik', keep="first")]
    Nodes=Nodes.drop(Minus.index).reset_index(drop=True)
  
    duplicates_company_name=Nodes[Nodes.duplicated('companyName',keep=False)].sort_values("companyName") #can I change them manually in Holdings,Filings and then make relationships, nodes?

    duplicates_company_name["companyName"]=duplicates_company_name["companyName"].str.cat(duplicates_company_name[["cik"]].astype(str), sep="_")
    
    Minus=duplicates_company_name
    Nodes=Nodes.drop(Minus.index).reset_index(drop=True)
    Nodes=pd.concat((Nodes,duplicates_company_name), axis=0, ignore_index=True)
    
    return Nodes,duplicates_company_name

def Relationships_(Holdings):
    Relationships=Holdings[["cusip","cik_Filing","titleOfClass","putCall","CIK","investmentDiscretion","value","nameOfIssuer_unchanged"]].drop_duplicates()
    Relationships["CIK"]=Relationships["CIK"].astype(int).astype(str)
    Relationships["CIK"]=Relationships["CIK"].str.lstrip("0").str.rstrip("0")
    Relationships["cik_Filing"]=Relationships["cik_Filing"].astype(int).astype(str)
    Relationships["cik_Filing"]=Relationships["cik_Filing"].str.lstrip("0").str.rstrip("0")
    return Relationships